# Individual Pages

In [5]:
def get_EBA_order_html_file(userid, pwd, eba_number):
    import requests
    from requests.packages.urllib3.exceptions import InsecureRequestWarning
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    payload = {'userid': userid, 'pwd': pwd,'eintragen':'++++login++++'}
    with requests.Session() as s:
        p = s.post('https://db.mbi-berlin.de/budget/ba_login.php', data=payload, verify=False)
        r = s.get('https://db.mbi-berlin.de/budget/ba_show.php?id=%d'%eba_number)
        if r.text == p.text:
            print('DID NOT WORK!')
            return None
        else:
            return r

def get_order_table_from_html(html):
    import os
    import sys
    import pandas as pd
    from bs4 import BeautifulSoup as bs

    data = []
    list_header = []
    soup = bs(html._content)
    # entries 1-3 are the header. Should make a function to get the header
    # entries 5+ are the stuff below the tables
    header = soup.find_all("table")[4].find("tr")
    
    for items in header:
        try:
            list_header.append(items.get_text())
        except:
            continue
            
    html_order_table = soup.find_all("table")[4].find_all("tr")[1:]
    
    return html_order_table

def parse_order_table(table):
    keys = [
    'Pos.Nr.',
    'Menge ME',
    'Bezeichnung der Lieferung u.Leistung',
    'Preis/ME',
    'Rabatt(%)',
    'Wert(Netto)',
    'Pos.-Typ',
    'Materialart/Mat.Nr.',
    'Sachkonto',
    'Finanzposition',
    'Fonds',
    'Kont.-Typ',
    'Artikel-Nr. des Lieferanten',
    'Kst/KTR/Auftrags-Nr.',
    'Finanzstelle',
    'Anlagen-Nr.'
    ]

    order = {}
    table_rows = table[3:-1]
    items_raw = [table_rows[i:i + 3] for i in range(0, len(table_rows), 3)]

    for n,item_raw in enumerate(items_raw):
        item = {}
        i = 0
        for item_data in item_raw:
            #print(item_data.get_text())
            for item_data_row in item_data:
                try:
                    if item_data_row.get_text() in ['\xa0','F\xa0\xa0']:
                        item[keys[i]] = ''
                        i += 1
                    else:
                        item[keys[i]] = item_data_row.get_text()
                        i += 1
                except:
                    continue
        order[n] = item
    return order

## Crawl

In [191]:
name = 'ENTER YOUR LDAP NAME HERE'
password = 'initial'
eba_id = 55498
#unique EBA id, consecutive. is not identical to the Bedards Nr
#Bedarfs Nr. are always between 50000 and 60000 and then they start over!
# As there are a total of 33703 orders, some Bedarfs Nr. are multiple times.
response = get_EBA_order_html_file(name,password,eba_id)
table = get_order_table_from_html(response)
table_formatted = parse_order_table(table)
table_formatted

{0: {'Pos.Nr.': '1',
  'Menge ME': '6 ST',
  'Bezeichnung der Lieferung u.Leistung': 'Adapter Sub-D 15Pin Stecker/Buchse',
  'Preis/ME': '2,10',
  'Rabatt(%)': '',
  'Wert(Netto)': '12,60',
  'Pos.-Typ': '',
  'Materialart/Mat.Nr.': '',
  'Sachkonto': '476100',
  'Finanzposition': '51502010',
  'Fonds': '',
  'Kont.-Typ': '',
  'Artikel-Nr. des Lieferanten': 'DELOCK 65481',
  'Kst/KTR/Auftrags-Nr.': '305343001',
  'Finanzstelle': '205343',
  'Anlagen-Nr.': ''},
 1: {'Pos.Nr.': '2',
  'Menge ME': '20 ST',
  'Bezeichnung der Lieferung u.Leistung': 'D-SUB-Buchse, 15-polig, Lötkelch, weiß',
  'Preis/ME': '0,22',
  'Rabatt(%)': '',
  'Wert(Netto)': '4,40',
  'Pos.-Typ': '',
  'Materialart/Mat.Nr.': '',
  'Sachkonto': '476100',
  'Finanzposition': '51502010',
  'Fonds': '',
  'Kont.-Typ': '',
  'Artikel-Nr. des Lieferanten': 'D-SUB BU 15 WS',
  'Kst/KTR/Auftrags-Nr.': '305343001',
  'Finanzstelle': '205343',
  'Anlagen-Nr.': ''},
 2: {'Pos.Nr.': '3',
  'Menge ME': '1 ST',
  'Bezeichnung der 

# Overview Table

In [185]:
def get_EBA_overview_html_file(userid, pwd, start_nr):
    import requests
    from requests.packages.urllib3.exceptions import InsecureRequestWarning
    requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
    payload = {'userid': userid, 'pwd': pwd,'eintragen':'++++login++++'}
    with requests.Session() as s:
        p = s.post('https://db.mbi-berlin.de/budget/ba_login.php', data=payload, verify=False)
        
        if start_nr == 0:
            r = s.get('https://db.mbi-berlin.de/budget/ba_select.php?&s=p')
        else:
            r = s.get('https://db.mbi-berlin.de/budget/ba_select.php?ListStart=%d&s=p'%start_nr)
        if r.text == p.text:
            print('DID NOT WORK!')
            return None
        else:
            return r
def get_orders(html):
    from bs4 import BeautifulSoup as bs
    soup = bs(html._content)
    # entries 1-3 are the header. Should make a function to get the header
    # entries 5+ are the stuff below the tables
    html_table = soup.find_all("table")

    keys = [
    'Bedarfs-Nr.',
    'Lieferant',
    'Anforderungsdat.',
    'UserID',
    'Status',
    'Nr.',
    'Art.Nr.',
    'Bezeichnung',
    'Finanzstelle',
    'Wert',
    ]

    orders = {}
    for entry in html_table[1].find_all("tr")[3:]:

        idnr = entry.find_all("a")[0]['href'].split('id=')[1].split('&')[0]
        orders[idnr] = {}
        i = 0
        for line in entry:

            try:
                orders[idnr][keys[i]] = line.get_text()
                i +=1
            except:
                pass
    return orders

## Crawl

In [188]:
page_nrs = arange(0,33704,200)
name = 'ENTER YOUR LDAP NAME HERE'
password = 'initial'
start_nr = 200
html = get_EBA_overview_html_file(name,password,start_nr)
orders = get_orders(html)
orders

{'55498': {'Bedarfs-Nr.': '53320',
  'Lieferant': 'Reichelt Elektronik GmbH & Co. KG',
  'Anforderungsdat.': '06.04.2022 11:09:00',
  'UserID': 'rohloff',
  'Status': 'Freigabe Verwaltung',
  'Nr.': '18',
  'Art.Nr.': 'Versandkosten',
  'Bezeichnung': 'Versand',
  'Finanzstelle': '205343',
  'Wert': '5,00'},
 '55496': {'Bedarfs-Nr.': '53319',
  'Lieferant': 'Vab Vakuum-Anlagenbau GmbH',
  'Anforderungsdat.': '06.04.2022 08:59:00',
  'UserID': 'korff',
  'Status': 'Freigabe Verwaltung',
  'Nr.': '1',
  'Art.Nr.': 'L-HE 2500',
  'Bezeichnung': 'He-Heber (L-Heber)für VAb-Verdampferkryostat',
  'Finanzstelle': '205343',
  'Wert': '2.610,00'},
 '55494': {'Bedarfs-Nr.': '53317',
  'Lieferant': 'Bartsch GmbH',
  'Anforderungsdat.': '06.04.2022 07:43:00',
  'UserID': 'sato',
  'Status': 'Freigabe Verwaltung',
  'Nr.': '1',
  'Art.Nr.': 'A190141BS',
  'Bezeichnung': 'Kaffeemaschine Pro 40T, 6L',
  'Finanzstelle': '205611',
  'Wert': '51,00'},
 '55493': {'Bedarfs-Nr.': '53316',
  'Lieferant': 'R